In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, cross_val_score
#from alg7 import model

In [4]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier()

In [7]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()

In [2]:
from xgboost import XGBClassifier
model=XGBClassifier(eval_metric='mlogloss')

In [6]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [7]:
# Carga de datos ya procesados
X=np.load(PREPROCESSED_DATA+'binScale.npz')
train = X['arr_0']
label = X['arr_1']
test = X['arr_2']
X.close()

In [8]:
train.shape

(9125, 52)

In [9]:
param_grid={'max_iter':[75,100,150,200,300], 'max_leaf_nodes':[27,29,31,33],'learning_rate':[0.08,0.1,0.12]}
searcher = GridSearchCV(model, param_grid, n_jobs=4,verbose=15,cv=5)
search = searcher.fit(train,label)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    5.9s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    6.0s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    6.1s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   11.0s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   11.8s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   12.4s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   16.8s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   17.1s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:   17.4s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:   18.6s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   22.6s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   23.0s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:   23.3s
[Parallel(

[Parallel(n_jobs=4)]: Done 135 tasks      | elapsed:  6.0min
[Parallel(n_jobs=4)]: Done 136 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 138 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 139 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 140 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done 141 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done 142 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done 143 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done 144 tasks      | elapsed:  6.4min
[Parallel(n_jobs=4)]: Done 145 tasks      | elapsed:  6.4min
[Parallel(n_jobs=4)]: Done 146 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done 147 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done 148 tasks      | elapsed:  6.6min
[Parallel(n_jobs=4)]: Done 149 tasks      | elapsed:  6.6min
[Parallel(n_jobs=4)]: Done 150 tasks      | elapsed:  6.6min
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 270 tasks      | elapsed: 12.8min
[Parallel(n_jobs=4)]: Done 271 tasks      | elapsed: 12.8min
[Parallel(n_jobs=4)]: Done 272 tasks      | elapsed: 12.9min
[Parallel(n_jobs=4)]: Done 273 tasks      | elapsed: 12.9min
[Parallel(n_jobs=4)]: Done 274 tasks      | elapsed: 13.0min
[Parallel(n_jobs=4)]: Done 275 tasks      | elapsed: 13.0min
[Parallel(n_jobs=4)]: Done 276 tasks      | elapsed: 13.2min
[Parallel(n_jobs=4)]: Done 277 tasks      | elapsed: 13.2min
[Parallel(n_jobs=4)]: Done 278 tasks      | elapsed: 13.3min
[Parallel(n_jobs=4)]: Done 279 tasks      | elapsed: 13.3min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 13.5min
[Parallel(n_jobs=4)]: Done 281 tasks      | elapsed: 13.5min
[Parallel(n_jobs=4)]: Done 282 tasks      | elapsed: 13.6min
[Parallel(n_jobs=4)]: Done 283 tasks      | elapsed: 13.7min
[Parallel(n_jobs=4)]: Done 284 tasks      | elapsed: 13.8min
[Parallel(n_jobs=4)]: Done 285 tasks      | elapsed: 13.9min
[Parallel(n_jobs=4)]: Do

In [11]:
df=pd.DataFrame(search.cv_results_)[['param_learning_rate','param_max_iter','param_max_leaf_nodes','mean_test_score','rank_test_score']].sort_values(by='rank_test_score')
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,param_learning_rate,param_max_iter,param_max_leaf_nodes,mean_test_score,rank_test_score
13,0.08,200,29,0.929973,1
45,0.12,100,29,0.929644,2
16,0.08,300,27,0.929644,2
17,0.08,300,29,0.929534,4
12,0.08,200,27,0.929534,4
58,0.12,300,31,0.929425,6
49,0.12,150,29,0.929205,7
54,0.12,200,31,0.929205,8
50,0.12,150,31,0.929205,8
33,0.1,200,29,0.929096,10


In [ ]:
param_grid={'n_estimators':[150,200,300,400,500], 'max_depth':[3,8,14,26,None]}
searcher = GridSearchCV(model, param_grid, n_jobs=4,verbose=15,cv=5)
search = searcher.fit(train,label)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
df=pd.DataFrame(search.cv_results_)[['param_learning_rate','param_n_estimators','param_num_leaves','mean_test_score','rank_test_score']].sort_values(by='rank_test_score')
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

In [14]:
param_grid={'learning_rate':[0.1,0.105,0.11],'n_estimators':[125,150,200,250],'num_leaves':[29,31]}
searcher = GridSearchCV(model, param_grid, n_jobs=4,verbose=15,cv=5)
search = searcher.fit(train,label)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    5.6s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    5.8s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    8.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    8.2s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   10.9s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:   11.3s
[Parallel(

In [16]:
df=pd.DataFrame(search.cv_results_)[['param_learning_rate','param_n_estimators','param_num_leaves','mean_test_score','rank_test_score']].sort_values(by='rank_test_score')
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,param_learning_rate,param_n_estimators,param_num_leaves,mean_test_score,rank_test_score
16,0.11,125,29,0.929096,1
22,0.11,250,29,0.929096,2
15,0.105,250,31,0.928877,3
13,0.105,200,31,0.928767,4
11,0.105,150,31,0.928767,4
5,0.1,200,31,0.928658,6
4,0.1,200,29,0.928658,6
20,0.11,200,29,0.928438,8
2,0.1,150,29,0.928438,9
3,0.1,150,31,0.928329,10


In [ ]:
print('Mejores parámetros:', search.best_params_)
print('Mejor puntuación:', search.best_score_)
model.set_params(**(search.best_params_))

In [ ]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

In [ ]:
model.fit(train,label)